In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [8]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [9]:
dependent=dataset[['charges']]

In [10]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [12]:
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
x_train=scx.fit_transform(x_train)
x_test=scx.transform(x_test)

In [13]:
scy=StandardScaler()
y_train=scy.fit_transform(y_train)
y_test=scy.transform(y_test)

In [14]:
x_train

array([[-1.5330973 , -0.40713453, -0.89833872, -0.97676557, -0.50466988],
       [-0.03364163,  0.32855417, -0.89833872, -0.97676557, -0.50466988],
       [ 0.89459283,  2.56690911,  3.25603402, -0.97676557, -0.50466988],
       ...,
       [ 0.03776102, -0.91016269, -0.89833872,  1.02378711, -0.50466988],
       [-1.46169465,  0.76659782, -0.89833872,  1.02378711, -0.50466988],
       [-0.46205754, -1.96596021, -0.06746417, -0.97676557, -0.50466988]])

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
param_grid = {'criterion':["squared_error","absolute_error","friedman_mse","poisson"],
'max_features': ["sqrt","log2",None],
'splitter':["best","random"]}
grid=GridSearchCV(DecisionTreeRegressor(),param_grid,refit=True,cv=5,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\Users\user\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 179, in fit
    "Some value(s) of y are negative which is"
ValueError: Some value(s) of y are negative 

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'friedman_mse', 'poisson'],
                         'max_features': ['sqrt', 'log2', None],
                         'splitter': ['best', 'random']},
             verbose=3)

In [16]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test)
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score)


The R_score value for best parameter {'criterion': 'friedman_mse', 'max_features': None, 'splitter': 'best'}: 0.7116162482891384


In [17]:
table=pd.DataFrame.from_dict(re)

In [18]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003125,6.250191e-03,0.000000,0.000000,squared_error,sqrt,best,"{'criterion': 'squared_error', 'max_features':...",0.708240,0.668623,0.615244,0.565057,0.653954,0.642224,0.048734,5
1,0.000000,0.000000e+00,0.006249,0.007654,squared_error,sqrt,random,"{'criterion': 'squared_error', 'max_features':...",0.522444,0.571525,0.562543,0.417861,0.607051,0.536285,0.065046,18
2,0.006256,7.661780e-03,0.000000,0.000000,squared_error,log2,best,"{'criterion': 'squared_error', 'max_features':...",0.476473,0.590760,0.722831,0.517101,0.584634,0.578360,0.083903,15
3,0.003125,6.249332e-03,0.000000,0.000000,squared_error,log2,random,"{'criterion': 'squared_error', 'max_features':...",0.693444,0.590581,0.625614,0.565996,0.531941,0.601515,0.055233,11
4,0.000000,0.000000e+00,0.000000,0.000000,squared_error,None,best,"{'criterion': 'squared_error', 'max_features':...",0.702651,0.543818,0.754990,0.638079,0.654977,0.658903,0.070484,2
5,0.003125,6.249809e-03,0.000000,0.000000,squared_error,None,random,"{'criterion': 'squared_error', 'max_features':...",0.756448,0.624197,0.708550,0.536112,0.634858,0.652033,0.075636,3
6,0.018749,6.249785e-03,0.003125,0.006250,absolute_error,sqrt,best,"{'criterion': 'absolute_error', 'max_features'...",0.585393,0.551132,0.654986,0.661540,0.644533,0.619517,0.043568,9
7,0.021874,7.655648e-03,0.000000,0.000000,absolute_error,sqrt,random,"{'criterion': 'absolute_error', 'max_features'...",0.588876,0.520440,0.542041,0.431209,0.669485,0.550410,0.078531,16
8,0.015626,9.536743e-07,0.000000,0.000000,absolute_error,log2,best,"{'criterion': 'absolute_error', 'max_features'...",0.661162,0.681944,0.498654,0.663326,0.653928,0.631803,0.067213,8
9,0.018751,6.250525e-03,0.000000,0.000000,absolute_error,log2,random,"{'criterion': 'absolute_error', 'max_features'...",0.476083,0.588956,0.644061,0.557254,0.651261,0.583523,0.064057,13


In [19]:
import pickle
filename="finalized_model_Decisiontree.sav"
filename1="scx.pkl"
filename2="scy.pkl"
pickle.dump(grid,open(filename,'wb'))
pickle.dump(scx,open(filename1,'wb'))
pickle.dump(scy,open(filename2,'wb'))

In [20]:
loaded_model=pickle.load(open("finalized_model_Decisiontree.sav",'rb'))
scx=pickle.load(open("scx.pkl",'rb'))
scy=pickle.load(open("scy.pkl",'rb'))

In [21]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age:32
BMI:37
Children:1
Sex Male 0 or 1:0
Smoker Yes 0 or 1:1


In [22]:
Preinput = scx.transform([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])

C:\Users\user\Anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [23]:
Future_Prediction=grid.predict(Preinput)

print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[2.14574645]


In [24]:
preoutput=scy.inverse_transform([Future_Prediction])

In [25]:
preoutput

array([[38711.]])